<a href="https://colab.research.google.com/github/nicholasfm/IAP2TXT/blob/master/IAP2TXT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt install tesseract-ocr
!suudo apt install poppler-utils
!pip install pytesseract
!pip install pdf2image

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 25 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (4,725 kB/s)
debconf: unable to initi

In [0]:
# Import libraries 
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os
#import urllib
import re
import shutil
import urllib.request as request
from contextlib import closing
from glob import glob
from google.colab import drive
import urllib

## Helper functions

In [0]:
def url_list_from_html(html):
    '''
    Use a regex pattern to find only the URLs (links) in some html.
    '''
    return re.findall(r"""<\s*a\s*href=["']([^=]+)["']""", html.decode("utf-8"))

def ensure_dir_exists(path):
    '''
    Create directory structure given by path if it doesn't already exist
    '''
    if not os.path.exists(path):
        os.makedirs(path)

def get_local_IAPs(local_IAP_dir,suffix_pattern='*.pdf'):
    available_IAPs = []
    for incident in glob(os.path.join(local_IAP_dir,'*')):
        available_IAPs.extend(glob(os.path.join(local_IAP_dir,incident,suffix_pattern)))
        #for incident_date in glob(os.path.join(incident,'*')):
        #    available_IAPs.extend(glob(os.path.join(incident_date,suffix_pattern)))
    return available_IAPs

def download_file(download_url,download_path,overwrite=False):
    already_exists = os.path.isfile(download_path)
    if not already_exists or already_exists and overwrite:
        try:
            with closing(request.urlopen(download_url)) as r:
                with open(download_path, 'wb') as f:
                    shutil.copyfileobj(r, f)
        except Exception as e:
            print('Download failed for {} with execption:'.format(download_url))
            print(e)
        print('Downloaded {}'.format(download_path))
    else:
        print('File {} already exists'.format(download_path))

In [7]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Download Params

In [0]:
region_agency = 'calif_n/!CALFIRE/'
base_url = 'https://ftp.nifc.gov/public/incident_specific_data/{}'.format(region_agency)
State_Code = 'CA'
download_corrected_IAPs = False
local_IAP_dir = '/content/drive/My Drive/IAPs/{}{}'.format(region_agency,State_Code)
ensure_dir_exists(local_IAP_dir)

In [9]:
local_IAP_dir

'/content/drive/My Drive/IAPs/calif_n/!CALFIRE/CA'

In [14]:
with urllib.request.urlopen(base_url) as response:
    urllist_year = url_list_from_html(response.read())
    
year_urls = [s for s in urllist_year if "_Incidents" in s]
    
for year_url in year_urls:
    year_total_IAPs = 0
    with urllib.request.urlopen(base_url + year_url) as response:
        urllist_incidents = url_list_from_html(response.read())
        
    # Get list of incidents using state code (first two chars in incident code) and exclude templates using 'xxx' pattern
    available_incidents = [s for s in urllist_incidents if State_Code == s[:2] and s[3:6] != 'xxx']
    for incident_url in available_incidents:

        try:
            with urllib.request.urlopen(base_url + year_url + incident_url) as response:
                available_urls_incident = url_list_from_html(response.read())

            IAP_urls = [s for s in available_urls_incident if 'IAP/' in s or 'IAPs/' in s or 'IAP\'s/' in s ]

            assert len(IAP_urls) == 1, 'There appears to be no IAP directory for {}'.format(base_url + year_url + incident_url)
            base_year_indident_IAP_url = base_url + year_url + incident_url + IAP_urls[0]
            
            with urllib.request.urlopen(base_year_indident_IAP_url) as response:
                available_urls_in_IAP_dir = url_list_from_html(response.read())

            #print(available_urls_in_IAP_dir)
            urls_to_download = []
            paths_for_downloading = []

            #First see if there's any PDFs in the top level of the IAP directory (common on older fires)
            pdf_list = [s for s in available_urls_in_IAP_dir if s[-4:] == '.pdf']
            if len(pdf_list) > 0:
                urls_to_download.extend([base_year_indident_IAP_url + pdfstring for pdfstring in pdf_list])
                ensure_dir_exists(os.path.join(local_IAP_dir, incident_url))
                paths_for_downloading.extend([os.path.join(local_IAP_dir, incident_url, pdfstring) for pdfstring in pdf_list])
                #Check to ensure we're downloading the right files with the right names (paths)
                assert len(urls_to_download) == len(paths_for_downloading), 'No. of URLs to download does not match no. of paths for those downloads for {}'.format(base_year_indident_IAP_url)
                print('PDFs were added from outside of date directories at {}'.format(base_year_indident_IAP_url))
                no_date_dir_PDFs = len(pdf_list)
            else:
                no_date_dir_PDFs = 0

            #Next go through any date directories in the IAP dir, if they exist
            #Logic is a little complicated as navigating ftp site via http doesn't allow us to check if a url is to a directory or file.
            #Basically checks if the date follows the format mm_dd_yy or mm-dd-yy by checking for the dash pattern, and excluding those that have the .pdf suffix
            IAP_date_urls = [s for s in available_urls_in_IAP_dir if s[2] == '-' and s[5] == '-' and s[-4:] != '.pdf' or s[2] == '_' and s[5] == '_' and s[-4:] != '.pdf']
            if len(IAP_date_urls) > 0:
                #Go through each of these directory links
                for IAP_date_url in IAP_date_urls:
                    with urllib.request.urlopen(base_year_indident_IAP_url + IAP_date_url) as response:
                        pdf_list = [s for s in url_list_from_html(response.read()) if s[-4:] == '.pdf']
                        if len(pdf_list) > 0:
                            urls_to_download.extend([base_year_indident_IAP_url + IAP_date_url + pdfstring for pdfstring in pdf_list])
                            ensure_dir_exists(os.path.join(local_IAP_dir, incident_url))
                            paths_for_downloading.extend([os.path.join(local_IAP_dir, incident_url, pdfstring) for pdfstring in pdf_list])
                            #Check to ensure we're downloading the right files with the right names (paths)
                            assert len(urls_to_download) == len(paths_for_downloading), 'No. of URLs to download does not match no. of paths for those downloads for {}'.format(base_year_indident_IAP_url + IAP_date_url)
                            date_dir_PDFs = len(pdf_list)
                        else:
                            print('No PDFs found in date directory at {}'.format(base_year_indident_IAP_url + IAP_date_url))
            else:
                date_dir_PDFs = 0


            for download_url, download_path in zip(urls_to_download,paths_for_downloading):
                download_file(download_url,download_path)


            total_incident_IAPs = no_date_dir_PDFs + date_dir_PDFs
            print('Downloaded {} IAPs for incident: {} in dated directories, {} without an date directory'.format(total_incident_IAPs, date_dir_PDFs, no_date_dir_PDFs))
            year_total_IAPs = year_total_IAPs + total_incident_IAPs
        except Exception as e:
            print('Could not download any IAPs for incident {}'.format(year_url.strip('!/').replace('_',' ') + incident_url.strip('/').replace('_',' ')))
            print(e)
    
    print('Downloaded {} {} IAPs'.format(year_total_IAPs,year_url.strip('!/').replace('_',' ')))

    

Downloaded 0 2020 Incidents IAPs
Could not download any IAPs for incident 2013 IncidentsCA-BTU-005638-Cedar
There appears to be no IAP directory for https://ftp.nifc.gov/public/incident_specific_data/calif_n/!CALFIRE/2013_Incidents/CA-BTU-005638-Cedar/
PDFs were added from outside of date directories at https://ftp.nifc.gov/public/incident_specific_data/calif_n/!CALFIRE/2013_Incidents/CA-BTU-005648-Panther/IAPs/
File /content/drive/My Drive/IAPs/calif_n/!CALFIRE/CA/CA-BTU-005648-Panther/IAP_5_4_2013.pdf already exists
File /content/drive/My Drive/IAPs/calif_n/!CALFIRE/CA/CA-BTU-005648-Panther/IAP_5_5_2013.pdf already exists
File /content/drive/My Drive/IAPs/calif_n/!CALFIRE/CA/CA-BTU-005648-Panther/IAP_5_6_2013.pdf already exists
File /content/drive/My Drive/IAPs/calif_n/!CALFIRE/CA/CA-BTU-005648-Panther/Panther%20Extended_IAP.pdf already exists
File /content/drive/My Drive/IAPs/calif_n/!CALFIRE/CA/CA-BTU-005648-Panther/iap%205-8-13.pdf already exists
File /content/drive/My Drive/IAPs/

'_test'

In [0]:
# Check if the uncorrected or corrected IAPs should be downloaded. 
if download_corrected_IAPs:
    valid_pdf_list = [s for s in pdf_list if 'CORRECTED' in s]
else:
    valid_pdf_list = [s for s in pdf_list if 'CORRECTED' not in s]

yes


## Convert downloaded IAP PDFs to text

In [0]:
available_IAPs = get_local_IAPs(local_IAP_dir)

In [0]:
available_IAPs

['./IAPs/norcal/incidents/CA-LNU-009502_County/07-04-2018/COUNTY_IAP_07-04-18.pdf',
 './IAPs/norcal/incidents/CA-LNU-009502_County/07-08-2018/COUNTY_IAP_07-08-2018.pdf',
 './IAPs/norcal/incidents/CA-LNU-009502_County/07-09-2018/COUNTY_IAP_07-09-18%20Run.pdf',
 './IAPs/norcal/incidents/CA-LNU-009502_County/07-05-2018/COUNTY_Corrected_7-5-18.pdf',
 './IAPs/norcal/incidents/CA-LNU-009502_County/07-02-2018/COUNTY_IAP_07-02-2018_reduced.pdf',
 './IAPs/norcal/incidents/CA-LNU-009502_County/07-03-2018/20180703_County_IAP_reduced.pdf',
 './IAPs/norcal/incidents/CA-LNU-009502_County/07-12-2018_3-DAY/COUNTY_3-DAY_07-12-18_3_Optimized.pdf',
 './IAPs/norcal/incidents/CA-LNU-009502_County/07-01-2018/COUNTY_IAP_07-01-2018.pdf',
 './IAPs/norcal/incidents/CA-LNU-009502_County/07-10-2018/COUNTY_IAP_07-10-18%20Run.pdf',
 './IAPs/norcal/incidents/CA-LNU-009502_County/07-06-2018/COUNTY_IAP_07-06-18.pdf',
 './IAPs/norcal/incidents/CA-LNU-009502_County/07-07-2018/COUNTY_IAP_07-07-18.pdf',
 './IAPs/norcal/in

In [0]:
for IAP_path in available_IAPs:
    iap2txt(IAP_path)

Completed conversion of ./IAPs/norcal/incidents/CA-LNU-009502_County/07-04-2018/COUNTY_IAP_07-04-18.txt
Completed conversion of ./IAPs/norcal/incidents/CA-LNU-009502_County/07-08-2018/COUNTY_IAP_07-08-2018.txt
Completed conversion of ./IAPs/norcal/incidents/CA-LNU-009502_County/07-09-2018/COUNTY_IAP_07-09-18%20Run.txt
Completed conversion of ./IAPs/norcal/incidents/CA-LNU-009502_County/07-05-2018/COUNTY_Corrected_7-5-18.txt
Completed conversion of ./IAPs/norcal/incidents/CA-LNU-009502_County/07-02-2018/COUNTY_IAP_07-02-2018_reduced.txt
Completed conversion of ./IAPs/norcal/incidents/CA-LNU-009502_County/07-03-2018/20180703_County_IAP_reduced.txt
Completed conversion of ./IAPs/norcal/incidents/CA-LNU-009502_County/07-12-2018_3-DAY/COUNTY_3-DAY_07-12-18_3_Optimized.txt
Completed conversion of ./IAPs/norcal/incidents/CA-LNU-009502_County/07-01-2018/COUNTY_IAP_07-01-2018.txt
Completed conversion of ./IAPs/norcal/incidents/CA-LNU-009502_County/07-10-2018/COUNTY_IAP_07-10-18%20Run.txt


In [0]:
def iap2txt(PDF_file):
    saved_image_paths, outpath = pdf2jpeg(PDF_file)
    jpeg2text(saved_image_paths, outpath)
    
def pdf2jpeg(PDF_file,overwrite_images=False):
    # Path of the pdf 
    #PDF_file = "./IAPs/incidents/IAP_KINCADE_10252019.pdf"
    basename = os.path.basename(PDF_file).split('.')
    iap_name = basename[0]
    outpath = os.path.join(os.path.dirname(PDF_file),iap_name+".txt")

    imagefile_dir = PDF_file[:-4]+'_page_images'
    ensure_dir_exists(imagefile_dir)
    

    ''' 
    Part #1 : Converting PDF to images 
    '''

    
    # Store all the pages of the PDF in a variable 
    pages = convert_from_path(PDF_file, 500) 

    # Counter to store images of each page of PDF to image 
    image_counter = 1
    saved_image_paths = []

    # Iterate through all the pages stored above 
    for page in pages: 

        # Declaring filename for each page of PDF as JPG 
        # For each page, filename will be: 
        # PDF page 1 -> page_1.jpg 
        # PDF page 2 -> page_2.jpg 
        # PDF page 3 -> page_3.jpg 
        # .... 
        # PDF page n -> page_n.jpg 
        #filename = iap_name+"_page_"+str(image_counter)+".jpg"
        filepath = os.path.join(imagefile_dir,iap_name+"_page_"+str(image_counter)+".jpg")
        
        # Save the image of the page in system 
        saved_image_paths.append(filepath)
        
        if not os.path.isfile(filepath):
            page.save(filepath, 'JPEG') 
        elif overwrite_images:
            page.save(filepath, 'JPEG')

        # Increment the counter to update filename 
        image_counter = image_counter + 1
        
    return saved_image_paths, outpath

def jpeg2text(saved_image_paths,outpath):

    ''' 
    Part #2 - Recognizing text from the images using OCR 
    '''
    # Variable to get count of total number of pages 
    #filelimit = image_counter-1

    # Creating a text file to write the output 
    #outfile = iap_name+".txt"
    
    # Open the file in append mode so that 
    # All contents of all images are added to the same file 
    #f = open(outfile, "a") 
    with open(outpath,"a") as file:

        # Iterate from 1 to total number of pages
        for image_path in saved_image_paths:
        #for i in range(1, filelimit + 1): 

            # Set filename to recognize text from 
            # Again, these files will be: 
            # page_1.jpg 
            # page_2.jpg 
            # .... 
            # page_n.jpg 
            #filename = "page_"+str(i)+".jpg"

            # Recognize the text as string in image using pytesserct 
            text = str(((pytesseract.image_to_string(Image.open(image_path))))) 

            # The recognized text is stored in variable text 
            # Any string processing may be applied on text 
            # Here, basic formatting has been done: 
            # In many PDFs, at line ending, if a word can't 
            # be written fully, a 'hyphen' is added. 
            # The rest of the word is written in the next line 
            # Eg: This is a sample text this word here GeeksF- 
            # orGeeks is half on first line, remaining on next. 
            # To remove this, we replace every '-\n' to ''. 
            text = text.replace('-\n', '')	 

            # Finally, write the processed text to the file. 
            file.write(text) 

    # Close the file after writing all the text. 
    #f.close() 
    print('Completed conversion of {}'.format(outpath))
